In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from joblib import load
import numpy as np
import pandas as pd
import warnings
import json
from datetime import datetime

# Suppress pandas warnings for cleaner output
warnings.filterwarnings("ignore", category=UserWarning)

# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
#  1. DEFINE THE NEURAL NETWORK ARCHITECTURE
# -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
class RegressionNet(nn.Module):
    def __init__(self, input_features):
        super(RegressionNet, self).__init__()
        self.layer1 = nn.Linear(input_features, 128)
        self.dropout1 = nn.Dropout(0.3)
        self.layer2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(0.2)
        self.layer3 = nn.Linear(64, 32)
        self.output_layer = nn.Linear(32, 1)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = self.dropout1(x)
        x = F.relu(self.layer2(x))
        x = self.dropout2(x)
        x = F.relu(self.layer3(x))
        x = torch.sigmoid(self.output_layer(x))
        return x

# -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
#  2. LOAD THE TRAINED MODEL AND SUPPORTING ASSETS
# -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
try:
    scaler_X = load('arch_models/scaler_X_granular.joblib')
    scaler_y = load('arch_models/scaler_y_granular.joblib')
    df_unit_cost = pd.read_csv('Thesis Data - Achitectural Unit Cost.csv')
    cost_cols_to_clean = [
        'Quantity of plaster (sq.m.)', 'Quantity of glazed tiles (sq.m.)',
        'Painting masonry (sq.m.)', 'painting wood (sq.m.)',
        'painting metal (sq.m.)', 'Area of CHB 100mm (sq.m.)',
        'Area of CHB 150mm (sq.m.)'
    ]
    for col in cost_cols_to_clean:
        if col in df_unit_cost.columns:
            df_unit_cost[col] = df_unit_cost[col].astype(str).str.replace(',', '')
            df_unit_cost[col] = pd.to_numeric(df_unit_cost[col], errors='coerce')
    num_input_features = scaler_X.n_features_in_
    model = RegressionNet(input_features=num_input_features)
    model.load_state_dict(torch.load('arch_models/ann_granular_model.pth'))
    model.eval()
    print("✅ Model and necessary assets loaded successfully.")
except FileNotFoundError as e:
    print(f"❌ Error loading assets: {e}")
    exit()

# -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
#  3. CREATE THE PREDICTION FUNCTION
# -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
def predict_budget(quantities, storeys, classrooms):
    """Predicts the project budget based on user-provided architectural quantities."""
    avg_unit_costs = {
        'plaster_cost': df_unit_cost['Quantity of plaster (sq.m.)'].median(),
        'glazed_tiles_cost': df_unit_cost['Quantity of glazed tiles (sq.m.)'].median(),
        'painting_masonry_cost': df_unit_cost['Painting masonry (sq.m.)'].median(),
        'painting_wood_cost': df_unit_cost['painting wood (sq.m.)'].median(),
        'painting_metal_cost': df_unit_cost['painting metal (sq.m.)'].median(),
        'chb_100mm_cost': df_unit_cost['Area of CHB 100mm (sq.m.)'].median(),
        'chb_150mm_cost': df_unit_cost['Area of CHB 150mm (sq.m.)'].median()
    }
    estimated_costs = [
        quantities['plaster_qty'] * avg_unit_costs['plaster_cost'],
        quantities['glazed_tiles_qty'] * avg_unit_costs['glazed_tiles_cost'],
        quantities['painting_masonry_qty'] * avg_unit_costs['painting_masonry_cost'],
        quantities['painting_wood_qty'] * avg_unit_costs['painting_wood_cost'],
        quantities['painting_metal_qty'] * avg_unit_costs['painting_metal_cost'],
        quantities['chb_100mm_qty'] * avg_unit_costs['chb_100mm_cost'],
        quantities['chb_150mm_qty'] * avg_unit_costs['chb_150mm_cost']
    ]
    input_features = estimated_costs + [storeys, classrooms]
    final_features = np.array(input_features).reshape(1, -1)
    scaled_features = scaler_X.transform(final_features)
    input_tensor = torch.tensor(scaled_features, dtype=torch.float32)
    with torch.no_grad():
        scaled_prediction = model(input_tensor).numpy()
    log_prediction = scaler_y.inverse_transform(scaled_prediction)
    final_prediction = np.expm1(log_prediction).flatten()[0]
    return final_prediction

# -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
#  4. USER INPUT SYSTEM
# -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
def get_user_input():
    """Prompts the user to enter project details, using defaults from a JSON file."""
    CONFIG_FILE = 'default_project_values.json'
    try:
        with open(CONFIG_FILE, 'r') as f:
            defaults = json.load(f)
            print(f"✅ Loaded default values from {CONFIG_FILE}")
    except FileNotFoundError:
        print(f"⚠️ Warning: {CONFIG_FILE} not found. Using hardcoded default values.")
        defaults = {
            "project_info": {"storeys": 2, "classrooms": 4},
            "quantities": {
                "plaster_qty": 1800, "glazed_tiles_qty": 100, "painting_masonry_qty": 2000,
                "painting_wood_qty": 150, "painting_metal_qty": 100, "chb_100mm_qty": 180,
                "chb_150mm_qty": 400
            }
        }
    
    print("\n--- 🏗️  Project Budget Prediction System ---")
    print("Please enter the quantities for the following architectural aspects.")
    print("(Press Enter to use the default value shown)\n")

    def get_float_input(prompt, default):
        while True:
            try:
                value = input(f"{prompt} (default: {default}): ")
                return float(value) if value else default
            except ValueError:
                print("❌ Invalid input. Please enter a number.")
    
    quantities = {
        'plaster_qty': get_float_input("Quantity of plaster (sq.m.)", defaults['quantities']['plaster_qty']),
        'glazed_tiles_qty': get_float_input("Quantity of glazed tiles (sq.m.)", defaults['quantities']['glazed_tiles_qty']),
        'painting_masonry_qty': get_float_input("Painting masonry (sq.m.)", defaults['quantities']['painting_masonry_qty']),
        'painting_wood_qty': get_float_input("painting wood (sq.m.)", defaults['quantities']['painting_wood_qty']),
        'painting_metal_qty': get_float_input("painting metal (sq.m.)", defaults['quantities']['painting_metal_qty']),
        'chb_100mm_qty': get_float_input("Area of CHB 100mm (sq.m.)", defaults['quantities']['chb_100mm_qty']),
        'chb_150mm_qty': get_float_input("Area of CHB 150mm (sq.m.)", defaults['quantities']['chb_150mm_qty']),
    }

    print("\n--- Project General Information ---")
    storeys = get_float_input("Number of Storeys", defaults['project_info']['storeys'])
    classrooms = get_float_input("Number of Classrooms", defaults['project_info']['classrooms'])
    
    return quantities, storeys, classrooms

# -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
#  5. LOGGING FUNCTION
# -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
def log_prediction_to_json(model_name, inputs, prediction):
    """Saves the inputs and prediction result to a JSON log file."""
    LOG_FILE = 'estimation_cost_logs.json'

    # *** FIX IS HERE ***: Convert the NumPy float32 to a standard Python float
    log_entry = {
        'model_name': model_name,
        'timestamp': datetime.now().isoformat(),
        'predicted_budget_php': round(float(prediction), 2), # <-- The fix
        'user_inputs': {
            'quantities': inputs['quantities'],
            'storeys': inputs['storeys'],
            'classrooms': inputs['classrooms']
        }
    }

    try:
        with open(LOG_FILE, 'r') as f:
            logs = json.load(f)
    except (FileNotFoundError, json.JSONDecodeError):
        logs = []

    logs.append(log_entry)

    try:
        with open(LOG_FILE, 'w') as f:
            json.dump(logs, f, indent=4)
        print(f"\n✅ Prediction successfully logged to {LOG_FILE}")
    except Exception as e:
        print(f"\n❌ Error: Could not write to log file. {e}")

# -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
#  6. MAIN EXECUTION BLOCK
# -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
if __name__ == "__main__":
    user_quantities, user_storeys, user_classrooms = get_user_input()
    predicted_budget = predict_budget(user_quantities, user_storeys, user_classrooms)
    print("\n-------------------------------------------")
    print("             PREDICTION RESULT             ")
    print("-------------------------------------------")
    print(f"💰 Estimated Project Budget: ₱{predicted_budget:,.2f}")
    print("\nDisclaimer: This is an estimate based on historical project data.")
    print("Actual costs may vary due to location, market conditions, and other factors.")
    print("-------------------------------------------")
    model_identifier = 'ann_granular_model_v1.0'
    all_inputs = {
        'quantities': user_quantities,
        'storeys': user_storeys,
        'classrooms': user_classrooms
    }
    log_prediction_to_json(model_identifier, all_inputs, predicted_budget)